# Effective correlation time analysis from database
### by: H. Antila, O. Ollila, H. Martinez-Seara

#### uses: MDAnalysis, Python3, gromacs5.1.4

## General Imports

In [2]:
# Working with files and directories
import os

#For quering webs
import urllib.request
from urllib.error import URLError,HTTPError

# From time monitoring
from tqdm import tqdm

# Python program to find SHA256 hash string of a file
import hashlib

# For dealing with excel and cvs 
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

#To make real independent copies of lists
from copy import deepcopy

## Directories

In [3]:
# Working directory
#dir_wrk  = "/usr/data/bgfs1/antila/teff_notebook"
dir_wrk="/usr/home/antila/teff_notebook/"
dir_wrk = os.path.normpath(dir_wrk)
print(dir_wrk)
def_file  = "/usr/data/bgfs1/antila/teff_notebook/POPC_lipid14_short.def"
def_file = os.path.normpath(def_file)
print(def_file)

/usr/home/antila/teff_notebook
/usr/data/bgfs1/antila/teff_notebook/POPC_lipid14_short.def


## Simulation description file

In [4]:
DOI="10.5281/zenodo.3335503"

user_information = """
ECC-POPC test
#NMRLIPIDS BEGIN

@SIM
@SOFTWARE=gromacs
@FF=ECC-POPC
@FF_SOURCE=Melcr et al.
@FF_DATE=19/12/2017
@TRJ=sim_ECC-POPC_SPCE_noIons-ref_300K.xtc
@TPR=sim_ECC-POPC_SPCE_noIons-ref_300K.tpr


@SIM
@SOFTWARE=gromacs
@FF=ECC-POPC
@FF_SOURCE=Melcr et al.
@FF_DATE=19/12/2017
@TRJ=sim_ECC-POPC_105mM_CaCl-2s_SPCE.xtc
@TPR=sim_ECC-POPC_105mM_CaCl-2s_SPCE.tpr


#NMRLIPIDS END

bla
sjss
ssd sskkd
"""

## Check that the DOI link is valid

In [5]:
DOI_url = 'https://doi.org/' + DOI
print(DOI_url)

try:
    response = urllib.request.urlopen(DOI_url)
    print("Status of the DOI link: {0}".format(response.msg))
except HTTPError as e:
    print(DOI_url)
    print('The server couldn\'t fulfill the request.')
    print('Error code: ', e.code)
    user_information = ""
    print('The code will not proceed, please fix DOI')
except URLError as e:
    print(DOI_url)
    print('We failed to reach a server.')
    print('Reason: ', e.reason)
    user_information = ""
    print('The code will not proceed, please fix DOI')
else:
    pass

https://doi.org/10.5281/zenodo.3335503
Status of the DOI link: OK


## Read input description

In [6]:
bNMRLIPIDS = False #Check if the link contains NMRLIPIDS metadata
nsims =0 # Counter number of simulations in a submission
sims = [] #Array with the dictionary containing the information of a simulation

for line in user_information.split("\n"):
    if line.strip() == "":
        continue
    if "#NMRLIPIDS BEGIN" in line:
        bNMRLIPIDS = True
        continue
    if "#NMRLIPIDS END" in line:
        bNMRLIPIDS = False
        continue
    if "@SIM" in line:
        #sims.append({"ID" : nsims, "STATUS" : 0})
        sims.append({"ID" : nsims})
        nsims += 1
        continue
    if not bNMRLIPIDS:
        continue
    if line.strip()[0] == "@":
        key, value = line.split("=")
        sims[-1][key.strip('@')] = value
print(nsims)
print(sims)      
        

2
[{'ID': 0, 'SOFTWARE': 'gromacs', 'FF': 'ECC-POPC', 'FF_SOURCE': 'Melcr et al.', 'FF_DATE': '19/12/2017', 'TRJ': 'sim_ECC-POPC_SPCE_noIons-ref_300K.xtc', 'TPR': 'sim_ECC-POPC_SPCE_noIons-ref_300K.tpr'}, {'ID': 1, 'SOFTWARE': 'gromacs', 'FF': 'ECC-POPC', 'FF_SOURCE': 'Melcr et al.', 'FF_DATE': '19/12/2017', 'TRJ': 'sim_ECC-POPC_105mM_CaCl-2s_SPCE.xtc', 'TPR': 'sim_ECC-POPC_105mM_CaCl-2s_SPCE.tpr'}]


### Dictionares

In [7]:
# Gromacs
gromacs_dict = {
               'INI' : {"REQUIRED": False,
                        "TYPE" : "files",
                        "EXTENSION" : ("gro", "pdb",),
                       }, # Could be not needed in the future (tpr)
               'MDP' : {"REQUIRED": False,
                        "TYPE" : "file",
                        "EXTENSION" : ("mdp",),
                       }, # Could be not needed in the future (tpr)
               'TRJ' : {"REQUIRED": True,
                        "TYPE" : "files",
                        "EXTENSION" : ("xtc","trr",),
                       },
               'TPR' : {"REQUIRED": True,
                        "TYPE" : "file",
                        "EXTENSION" : ("tpr",),
                       },
               'CPT' : {"REQUIRED": False,
                        "TYPE" : "file",
                        "EXTENSION" : ("cpt",),
                       },
               'TOP' : {"REQUIRED": False,
                        "TYPE" : "file",
                        "EXTENSION" : ("top",),
                       },
               'ITP' : {"REQUIRED": False,
                        "TYPE" : "files",
                        "EXTENSION" : ("itp",),
                       },
               'FF'  : {"REQUIRED": False,
                        "TYPE" : "string",
                       },
               'FF_SOURCE' : {"REQUIRED": False,
                              "TYPE" : "string",
                              },
               'FF_DATE' : {"REQUIRED": False,
                            "TYPE" : "date",
                           },
               }

# Amber
amber_dict = {}

# NAMD
namd_dict = {}

# CHARMM
charmm_dict = {}

# OPENMM
openmm_dict = {}

# SOFTWARE
software_dict = {
                "GROMACS" : gromacs_dict, 
                "AMBER"   : amber_dict,
                "NAMD"    : namd_dict,
                "CHARMM"  : charmm_dict,
                "OPENMM"  : openmm_dict,
                }

print(software_dict.keys())

dict_keys(['GROMACS', 'AMBER', 'NAMD', 'CHARMM', 'OPENMM'])


### Check software used by the simulation

In [8]:
sims_valid_software = []
for sim in sims:
    if sim['SOFTWARE'].upper() in software_dict.keys():
        msg_info = "Simulation {0} uses supported software {1} and will be further processed"
        print (msg_info.format(sim['ID'], sim['SOFTWARE'].upper()))
        sims_valid_software.append(sim.copy())
    else:
        msg_err="Simulation {0} performed in an UNSUPPORTED software {1} and will NOT be further processed"
        print(msg_err.format(sim["ID"], sim["SOFTWARE"].upper()))
#print(sims_valid_software) 

Simulation 0 uses supported software GROMACS and will be further processed
Simulation 1 uses supported software GROMACS and will be further processed


### Check that all entry keys provided for each simulation are valid:

In [9]:
sims_valid_entries = []
for sim in sims_valid_software:
    #print("ID {0}".format(sim["ID"]))
    wrong_key_entries = 0
    software_dict_name = "{0}_dict".format(sim['SOFTWARE'].lower())
    for key_sim, value_sim in sim.items():
        #print(key_sim, value_sim)
        if key_sim.upper() in ("ID", "SOFTWARE"):
            #print("NOT REQUIRED")
            continue
        if key_sim.upper() not in software_dict[sim['SOFTWARE'].upper()].keys():
            print ("{0} NOT in {1}".format(key_sim, software_dict_name))
            wrong_key_entries += 1
    if wrong_key_entries:
        print("Simulation {0} has {1} unknown entry/ies and won't be longer considered, please correct.\n".format(sim['ID'],wrong_key_entries))
    else:
        msg_info = "All entries in simulation {0} are understood and will be further processed\n"
        print (msg_info.format(sim['ID']))
        sims_valid_entries.append(sim.copy())
#print(sims_valid_entries)

All entries in simulation 0 are understood and will be further processed

All entries in simulation 1 are understood and will be further processed



### Process entries with files information to contain file names in arrays

In [10]:
sims_files_to_array = deepcopy(sims_valid_entries)

for sim in sims_files_to_array:
    print("ID {0}".format(sim["ID"]), flush=True)
    software_sim = software_dict[sim['SOFTWARE'].upper()]
    for key_sim, value_sim in sim.items():
        try:
            entry_type = software_sim[key_sim]['TYPE']
            if "file" in entry_type:
                if isinstance(value_sim, list): continue  
                files_list = []
                print("{0} added to list".format(value_sim))
                # Place filenames into arrays
                for file_provided in value_sim.split(";"):
                    files_list.append([file_provided.strip()])
                sim[key_sim] = files_list
        except: #It is notmal that fails for "ID" and "SOFTWARE"
            continue
#print(sims_files_to_array)
#print(sims_valid_entries)

ID 0
sim_ECC-POPC_SPCE_noIons-ref_300K.xtc added to list
sim_ECC-POPC_SPCE_noIons-ref_300K.tpr added to list
ID 1
sim_ECC-POPC_105mM_CaCl-2s_SPCE.xtc added to list
sim_ECC-POPC_105mM_CaCl-2s_SPCE.tpr added to list


### Check for multiple files in entries that can only contain one

In [11]:
sims_valid_file_entries = []
for sim in sims_files_to_array:
    print("ID {0}".format(sim["ID"]), flush=True)
    files_issues = 0
    software_sim = software_dict[sim['SOFTWARE'].upper()]
    for key_sim, value_sim in sim.items():
        try:
            entry_type = software_sim[key_sim]['TYPE']
            if entry_type == "file"  and len(value_sim) > 1:
                print("Multiple values found in {0} and only one allowed (Please correct):\n {1}".format(key_sim,value_sim))
                files_issues += 1
        except: #It is notmal that fails for "ID" and "SOFTWARE"
            continue
    if files_issues:
        print("Sim {0} will be no longer processed".format(sim["ID"]))
    else:
        sims_valid_file_entries.append(sim.copy())
#print(sims_valid_file_entries)

ID 0
ID 1


### Check if the submitted simulation has rssion has all required files and information

In [12]:
sims_required_entries = []
for sim in sims_valid_file_entries:
    print("ID {0}".format(sim["ID"]))
    missing_required_keys = 0
    for key, value in software_dict[sim['SOFTWARE'].upper()].items():
        if value["REQUIRED"]:
            try:
                sim[key]
            except:
                print("Entry not found: {0} {1}".format(key, value))
                missing_required_keys += 1
    if missing_required_keys:
        print("{0} missing required entry/ies, please correct.".format(missing_required_keys))
        print("Entry with ID={0} will not be further processed.\n".format(sim["ID"]))
    else:
        print("All required entries present.\n")
        sims_required_entries.append(sim.copy())

ID 0
All required entries present.

ID 1
All required entries present.



### Check status links

In [13]:
# Download link
def download_link(doi, file):
    if "zenodo" in doi.lower():
        zenodo_entry_number = doi.split(".")[2]
        return 'https://zenodo.org/record/' + zenodo_entry_number + '/files/' + file
    else:
        print ("DOI provided: {0}".format(doi))
        print ("Repository not validated. Please upload the data for example to zenodo.org")
        return ""

In [14]:
#print(sims_required_entries)
sims_working_links = []         
for sim in sims_required_entries:
    print("ID {0}".format(sim["ID"]))
    wrong_links = 0
    software_sim = software_dict[sim['SOFTWARE'].upper()]
    for key_sim, value_sim in sim.items():
        #print("key_sim = {0} => value_sim = {1}".format(key_sim, value_sim))
        try:
            entry_type = software_sim[key_sim]['TYPE']
            #print("entry_type = {0}".format(entry_type))
            if "file" in entry_type:
                for file_provided in value_sim:
                    #print("File={0}".format(file_provided[0]))
                    file_url = download_link(DOI, file_provided[0])
                    if file_url == "":
                        wrong_links += 1
                        continue
                    try:
                        response = urllib.request.urlopen(file_url)
                        #print("Status of the DOI link: {0}".format(response.msg))
                    except HTTPError as e:
                        print("\nkey={0} => file={1}".format(key_sim, file_provided[0]))
                        print(file_url)
                        print('The server couldn\'t fulfill the request.')
                        print('Error code: ', e.code)
                        wrong_links += 1
                    except URLError as e:
                        print(key_sim, file_provided[0])
                        print(file_url)
                        print('We failed to reach a server.')
                        print('Reason: ', e.reason)
                        wrong_links += 1
                    else:
                        pass
        except: #It is notmal that fails for "ID" and "SOFTWARE"
            continue
    if wrong_links:
        print("{0} link/s failed, please correct.".format(wrong_links))
        print("Sim={0} will not be further processed.\n".format(sim["ID"]))
    else:
        print("All links work.\n")
        sims_working_links.append(sim.copy())
#print(sims_working_links)

ID 0
All links work.

ID 1
All links work.



## Download files from links

In [15]:
# Create temporary directory where to download files and analyze them
dir_tmp = os.path.join(dir_wrk, "tmp/")
if (not os.path.isdir(dir_tmp)): os.mkdir(dir_tmp)

for sim in sims_working_links:
    print("ID {0}".format(sim["ID"]), flush=True)
    software_sim = software_dict[sim['SOFTWARE'].upper()]
    dir_sim = os.path.join(dir_tmp, str(sim["ID"]))
    
    if (not os.path.isdir(dir_sim)): os.mkdir(dir_sim)
    for key_sim, value_sim in sim.items():
        #print("key_sim = {0} => value_sim = {1}".format(key_sim, value_sim))
        try:
            entry_type = software_sim[key_sim]['TYPE']
            #print("entry_type = {0}".format(entry_type))
            if "file" in entry_type:
                for file_provided in tqdm(value_sim, desc = key_sim):
                    file_url = download_link(DOI, file_provided[0])
                    file_name = os.path.join(dir_sim, file_provided[0])
                    if (not os.path.isfile(file_name)):
                        response = urllib.request.urlretrieve(file_url, file_name)
        except: #It is notmal that fails for "ID" and "SOFTWARE"
            continue
            

ID 0


TPR: 100%|██████████| 1/1 [00:00<00:00, 3968.12it/s]

ID 1



TPR: 100%|██████████| 1/1 [00:00<00:00, 1987.82it/s]


## Calculate hash downloaded files

In [16]:
dir_tmp = os.path.join(dir_wrk, "tmp/")
sims_hashes = deepcopy(sims_working_links)

for sim in sims_hashes:
    print("ID {0}".format(sim["ID"]), flush=True)
    software_sim = software_dict[sim['SOFTWARE'].upper()]
    dir_sim = os.path.join(dir_tmp, str(sim["ID"]))
    
    #list_containing the sha1 sums for all required files
    sha1_list_requied = []
    
    # Make empty dataframe with the desired columns
    df_files = pd.DataFrame(columns=['NAME','TYPE','REQUIRED','HASH'])
    
    for key_sim, value_sim in sim.items():
        #print("key_sim = {0} => value_sim = {1}".format(key_sim, value_sim))
        try:
            entry_type = software_sim[key_sim]['TYPE']
            #print("entry_type = {0}".format(entry_type))
            if "file" in entry_type:
                files_list = []
                for file_provided in value_sim:
                    file_name = os.path.join(dir_sim, file_provided[0])
                    sha1_hash = hashlib.sha1()
                    with open(file_name,"rb") as f:
                        # Read and update hash string value in blocks of 4K
                        for byte_block in iter(lambda: f.read(4096),b""):
                            sha1_hash.update(byte_block)
                        #print(file_provided, sha256_hash.hexdigest())
                        df_files = df_files.append({
                            "NAME":file_provided[0],
                            "TYPE":key_sim,
                            "REQUIRED": software_dict[sim['SOFTWARE'].upper()][key_sim]['REQUIRED'],
                            "HASH":sha1_hash.hexdigest(),
                        }, ignore_index=True)
                    files_list.append([file_provided[0], sha1_hash.hexdigest()])
                #Find the keys of the required files to calculate the master_hash 
                if software_dict[sim['SOFTWARE'].upper()][key_sim]['REQUIRED'] == True:
                    sha1_list_requied.append(sha1_hash.hexdigest())
                sim[key_sim] = files_list #Problematic
        except: #It is notmal that fails for "ID" and "SOFTWARE"
            continue
    print(df_files)
    print("\n{0}\n".format(sha1_list_requied))      
    # Calculate the hash of a file contaning the hashes of each of the required files
    # This should be always invariant as it will be used unique identifier for a simualtion
    # Note order the hashes of the required files before calculating the hash (That means that the required files cannot change)
print(sims_hashes)

ID 0
                                    NAME TYPE REQUIRED                                      HASH
0  sim_ECC-POPC_SPCE_noIons-ref_300K.xtc  TRJ     True  5ad07cc3a737f917dc0e69d6f00b44091322ff68
1  sim_ECC-POPC_SPCE_noIons-ref_300K.tpr  TPR     True  9958771b8fb92ac07be7d4285d09f8df65cc758e

['5ad07cc3a737f917dc0e69d6f00b44091322ff68', '9958771b8fb92ac07be7d4285d09f8df65cc758e']

ID 1
                                  NAME TYPE REQUIRED                                      HASH
0  sim_ECC-POPC_105mM_CaCl-2s_SPCE.xtc  TRJ     True  ca17d550c76a9365915b4f054efe441d839425ab
1  sim_ECC-POPC_105mM_CaCl-2s_SPCE.tpr  TPR     True  620e7e473f437da500cb26f4283698b9979b9c3e

['ca17d550c76a9365915b4f054efe441d839425ab', '620e7e473f437da500cb26f4283698b9979b9c3e']

[{'ID': 0, 'SOFTWARE': 'gromacs', 'FF': 'ECC-POPC', 'FF_SOURCE': 'Melcr et al.', 'FF_DATE': '19/12/2017', 'TRJ': [['sim_ECC-POPC_SPCE_noIons-ref_300K.xtc', '5ad07cc3a737f917dc0e69d6f00b44091322ff68']], 'TPR': [['sim_ECC-POPC_SPCE_no

In [17]:
sims_working_links

[{'ID': 0,
  'SOFTWARE': 'gromacs',
  'FF': 'ECC-POPC',
  'FF_SOURCE': 'Melcr et al.',
  'FF_DATE': '19/12/2017',
  'TRJ': [['sim_ECC-POPC_SPCE_noIons-ref_300K.xtc']],
  'TPR': [['sim_ECC-POPC_SPCE_noIons-ref_300K.tpr']]},
 {'ID': 1,
  'SOFTWARE': 'gromacs',
  'FF': 'ECC-POPC',
  'FF_SOURCE': 'Melcr et al.',
  'FF_DATE': '19/12/2017',
  'TRJ': [['sim_ECC-POPC_105mM_CaCl-2s_SPCE.xtc']],
  'TPR': [['sim_ECC-POPC_105mM_CaCl-2s_SPCE.tpr']]}]

# Analysis starts here
## First dowload packages and calculate the correlation functions using gromacs tools

In [20]:
from OrderParameter import *
import warnings
from corrtimes import *
import subprocess
for sim in sims_working_links:
    trj=sim.get('TRJ')
    tpr=sim.get('TPR')
    ID=sim.get('ID')
    
    tprnowat='tmp/'+str(ID)+'/'+'nowater.tpr'
    xtcnowatwhole='tmp/'+str(ID)+'/'+'whole.xtc'
    #trj='tmp/'+str(ID)+'/'+str(trj[0][0])
    #tpr='tmp/'+str(ID)+'/'+str(tpr[0][0])
    #print(def_file,tpr,trj)


    #Hanne: removing the waters for faster analysis
    !echo non-water | gmx trjconv -f {dir_wrk}/tmp/{ID}/{str(trj[0][0])} -t0 0 -s {dir_wrk}/tmp/{ID}/{str(tpr[0][0])} -o {xtcnowatwhole} -pbc whole
   # !echo non-Water | gmx trjconv -f tmp.trr -s {dir_wrk}/tmp/{ID}/{str(tpr[0][0])} -o {dir_wrk}/tmp/{ID}/whole.xtc -pbc mol 
    !echo non-Water | gmx convert-tpr -s {dir_wrk}/tmp/{ID}/{str(tpr[0][0])} -o {tprnowat}
    !rm tmp.trr 

 

   
    
    output_corr=!sh {dir_wrk}/corr_ftios_ind_fast.sh {def_file} {tprnowat} {xtcnowatwhole} {dir_wrk}/tmp/{ID}
    #tmp=output_corr.split()
    #print (output_corr)
    tmp=output_corr[-1].split()
    
    firstlipid=int(tmp[-2])
    lastlipid=int(tmp[-1])
    numlipid=lastlipid-firstlipid+1    
    
    print (ID, numlipid)
    
    
    OrdParam=find_OP(def_file,tprnowat,xtcnowatwhole)
    #this file contains a op-name, op value, op error, R1, R1 error
    outfile=open('tmp/'+str(ID)+'/'+'times'+str(ID)+'R1.txt','w')
    #this file contains op-name, op value, teff average (-1 if not defined), teff_min, teff_max
    outfile_teffs=open('tmp/'+str(ID)+'/times'+str(ID)+'_teffs.txt','w')

    
    for i,op in enumerate(OrdParam.values()):
        resops =op.get_op_res
        (op.avg, op.std, op.stem) =op.get_avg_std_stem_OP
        
        #individual lipid staticstis, removed for now
        #outfile_ind=open('tmp/'+str(ID)+'/'+op.name+'times_perlipid.txt','w')
        convs=[]
        Teffs=[]
        Teffs_area=[]
        R1s=[]
        #this loop separates the correlation ftions when calc_corrf is run with noaver. 
        finc=open('tmp/'+str(ID)+'/'+'rotacf_'+op.name+'.xvg',"r")

        lines=finc.readlines()
        k=firstlipid
        foutc=open('tmp/'+str(ID)+'/'+str(k)+"_rotacf_"+op.name+".xvg","w")
        for line in lines:

            if "&" in line:
                k=k+1
                foutc.close()
 
                if k<=lastlipid:
                    foutc=open('tmp/'+str(ID)+'/'+str(k)+"_rotacf_"+op.name+".xvg","w")
            else:
                foutc.write(line)
        
        finc.close()
        
        for j in range(firstlipid,lastlipid+1):
            print (op.name, j)
            corrfile=dir_wrk+'/tmp/'+str(ID)+'/'+str(j)+'_rotacf_'+op.name+".xvg"
            if j==1:
                out,times=read_data(corrfile)
                fvals=np.asmatrix(out).T
            else:
                out, times=read_data(corrfile)
                fvals=np.concatenate((fvals,np.asmatrix(out).T),axis=1)
            #Hanne: could change to calc only R1 here
            Teff,tau_eff_area, R1, conv =calc_corrtime_noread(out,times, resops[j-1])
            Teffs.append(Teff)
            Teffs_area.append(tau_eff_area)
            R1s.append(R1)
            convs.append(conv)
            
         #statistics for idividual lipids, removed for now   
         #   line=str(op.name)+" "+str(resops[j-1])+" "+str(Teff)+" "+str(tau_eff_area)+" "+str(R1)+" "+str(conv)+ '\n'
         #   outfile_ind.write(line)
        #outfile_ind.close()

        line2=str(op.name)+" "+str(op.avg)+" "+str(op.stem)+" "+str(np.mean(R1s))+" "+str(np.std(R1s, ddof=1)/np.sqrt(len(R1s)-1))+" "+str(sum(convs)/float(len(convs)))+'\n'
        outfile.write(line2)

        #analysis of Teff error starts here
        means=np.mean(fvals,axis=1)
        stems=np.std(fvals,axis=1,ddof=1)/np.sqrt(int(numlipid)-1)
        teff,teff_min,teff_max=calc_corrtime_withee(times,means, stems, op.avg, op.stem)
        line3=str(op.name)+" "+str(op.avg)+" "+str(teff)+" "+str(teff_min)+" "+str(teff_max)+'\n'
        outfile_teffs.write(line3)        
      

    outfile_teffs.close()
    outfile.close()
    

                  :-) GROMACS - gmx trjconv, VERSION 5.1.4 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov  Herman J.C. Berendsen    Par Bjelkmar   
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra   Sebastian Fritsch 
  Gerrit Groenhof   Christoph Junghans   Anca Hamuraru    Vincent Hindriksen
 Dimitrios Karkoulis    Peter Kasson        Jiri Kraus      Carsten Kutzner  
    Per Larsson      Justin A. Lemkul   Magnus Lundborg   Pieter Meulenhoff 
   Erik Marklund      Teemu Murtola       Szilard Pall       Sander Pronk   
   Roland Schulz     Alexey Shvetsov     Michael Shirts     Alfons Sijbers  
   Peter Tieleman    Teemu Virolainen  Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess, Erik Lindahl, and David van der Spoel

Copyright (c) 1991-2000, University of Groningen, The Netherlands.
Copyright (c) 2001-2015, The GROMACS development team at
Uppsala Universit

alpha1 86
alpha1 87
alpha1 88
alpha1 89
alpha1 90
alpha1 91
alpha1 92
alpha1 93
alpha1 94
alpha1 95
alpha1 96
alpha1 97
alpha1 98
alpha1 99
alpha1 100
alpha1 101
alpha1 102
alpha1 103
alpha1 104
alpha1 105
alpha1 106
alpha1 107
alpha1 108
alpha1 109
alpha1 110
alpha1 111
alpha1 112
alpha1 113
alpha1 114
alpha1 115
alpha1 116
alpha1 117
alpha1 118
alpha1 119
alpha1 120
alpha1 121
alpha1 122
alpha1 123
alpha1 124
alpha1 125
alpha1 126
alpha1 127
alpha1 128
g1_2 1
g1_2 2
g1_2 3
g1_2 4
g1_2 5
g1_2 6
g1_2 7
g1_2 8
g1_2 9
g1_2 10
g1_2 11
g1_2 12
g1_2 13
g1_2 14
g1_2 15
g1_2 16
g1_2 17
g1_2 18
g1_2 19
g1_2 20
g1_2 21
g1_2 22
g1_2 23
g1_2 24
g1_2 25
g1_2 26
g1_2 27
g1_2 28
g1_2 29
g1_2 30
g1_2 31
g1_2 32
g1_2 33
g1_2 34
g1_2 35
g1_2 36
g1_2 37
g1_2 38
g1_2 39
g1_2 40
g1_2 41
g1_2 42
g1_2 43
g1_2 44
g1_2 45
g1_2 46
g1_2 47
g1_2 48
g1_2 49
g1_2 50
g1_2 51
g1_2 52
g1_2 53
g1_2 54
g1_2 55
g1_2 56
g1_2 57
g1_2 58
g1_2 59
g1_2 60
g1_2 61
g1_2 62
g1_2 63
g1_2 64
g1_2 65
g1_2 66
g1_2 67
g1_2 68
g1_2 6


gcq#437: "The researcher's art is first of all to find himself a good boss." (Andre Lwoff)

rm: cannot remove 'tmp.trr': No such file or directory
1 128
beta1 1
beta1 2
beta1 3
beta1 4
beta1 5
beta1 6
beta1 7
beta1 8
beta1 9
beta1 10
beta1 11
beta1 12
beta1 13
beta1 14
beta1 15
beta1 16
beta1 17
beta1 18
beta1 19
beta1 20
beta1 21
beta1 22
beta1 23
beta1 24
beta1 25
beta1 26
beta1 27
beta1 28
beta1 29
beta1 30
beta1 31
beta1 32
beta1 33
beta1 34
beta1 35
beta1 36
beta1 37
beta1 38
beta1 39
beta1 40
beta1 41
beta1 42
beta1 43
beta1 44
beta1 45
beta1 46
beta1 47
beta1 48
beta1 49
beta1 50
beta1 51
beta1 52
beta1 53
beta1 54
beta1 55
beta1 56
beta1 57
beta1 58
beta1 59
beta1 60
beta1 61
beta1 62
beta1 63
beta1 64
beta1 65
beta1 66
beta1 67
beta1 68
beta1 69
beta1 70
beta1 71
beta1 72
beta1 73
beta1 74
beta1 75
beta1 76
beta1 77
beta1 78
beta1 79
beta1 80
beta1 81
beta1 82
beta1 83
beta1 84
beta1 85
beta1 86
beta1 87
beta1 88
beta1 89
beta1 90
beta1 91
beta1 92
beta1 93
beta1 94
beta1 95
